In [89]:
import numpy as np
import scipy as sc

from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
from scipy.sparse import kron as tensor_product
from scipy.sparse.linalg import expm as matrix_exponential

In [92]:
class QuantumGate:
    def __init__(self, name: str, scaling: float = 1.0):
        self.name = name
        self.size = len(name)
        self.__scaling__ = scaling
        self.__matrix_representation__ = self._set_matrix_()
    
    def _set_matrix_(self):
        pass
    
    def size(self):
        return self.size

    def name(self):
        return self.name
    
    def matrix(self):
        return self.__scaling__ * self.__matrix_representation__
    
    def time_evolution(self, time: float=1.0) -> np.ndarray:
        return matrix_exponential(A=-1j*time*self.matrix())
    
        
class PauliGate(QuantumGate):
    def __init__(self, name: str, scaling: float = 1.0):
        self._X_ = csc_matrix(np.array([[0,1],[1,0]]   , dtype=np.complex64))
        self._Y_ = csc_matrix(np.array([[0,-1j],[1j,0]], dtype=np.complex64))
        self._Z_ = csc_matrix(np.array([[1,0],[0,-1]]  , dtype=np.complex64))
        self._I_ = csc_matrix(np.array([[1,0],[0,1]]   , dtype=np.complex64))
        self.__allowed_gates__ = {'x': self._X_, 'X': self._X_,
                                  'y': self._Y_, 'Y': self._Y_,
                                  'z': self._Z_, 'Z': self._Z_,
                                  'i': self._I_, 'I': self._I_}
        if len(name) == 1:
            if name not in list(self.__allowed_gates__.keys()):
                raise ValueError(f'Unknown gate name provided, '
                                 f'should be one: {list(self.__allowed_gates__.keys())}')
        elif len(name) > 1:
            for symbol in name:
                if symbol not in list(self.__allowed_gates__.keys()):
                    raise ValueError(f'Unknown gate name provided, '
                                     f'should be one: {list(self.__allowed_gates__.keys())}')
            
        super().__init__(name=name, scaling=scaling)
        
    def _set_matrix_(self):
        if len(self.name) == 1:
            return self.__allowed_gates__[self.name]
        elif len(self.name) > 1:
            resulting_tensor_product = self.__allowed_gates__[self.name[0]]
            for remaining_symbol in range(1,len(self.name)):
                resulting_tensor_product = tensor_product(resulting_tensor_product, self.__allowed_gates__[self.name[remaining_symbol]], format=resulting_tensor_product.format)
            return resulting_tensor_product

        
        
    
        

In [93]:
gate_name = 'xxxxxxxx'

obj = PauliGate(name=gate_name)